# Import Module

In [4]:
!pip install cohere

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import cohere
import time

# Data Cleaning and Processing

In [18]:
# We write a scan function to check for validation and consistent of datasets
def validation_check(df):
    print(f'Shape is: {df.shape} \n')
    print(f'Null values summary:')
    display(df.isnull().sum())
    print(f'\nTypes of data:')
    display(df.dtypes)

## 1.Table D_user 

In [19]:
try:
    df_user = pd.read_excel(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_user')
except: 
    df_user = pd.read_excel(r'C:\Users\hoang\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_user')

In [20]:
validation_check(df_user)

Shape is: (66717, 4) 

Null values summary:


gender           6248
user_id             0
province_name       0
age                 0
dtype: int64


Types of data:


gender           object
user_id          object
province_name    object
age              object
dtype: object

Currently, there are only null value in gender dimensions.
In here we will scan through 3 columns ['gender', 'province_name', 'age'] for any further in-valid value

In [21]:
print('Distinct values of each columns are:')
for col_name in ['gender', 'province_name', 'age']:
    print (f'{col_name}: {df_user[col_name].unique().tolist()} \n' ) 

Distinct values of each columns are:
gender: ['F', nan, 'M', 'unavailable'] 

province_name: ['Hà Nội', 'Miền Trung', 'Hồ Chí Minh', 'Miền Bắc', 'HÀ NỘI', 'UNIDENTIFIED', 'HCM', 'ĐN', 'Miền Nam', 'Hànoi', 'Đà Nẵng'] 

age: ['O55', '25_34', 'UNIDENTIFIED', 'U18', '18_24', '35_44', '45_54', '28', 43, 46, 40, 35, 44, 38, 47, 45, 37, 41, 36, 39, 42, 'Hai Tư'] 



There is some problems at each columns, such as: 
- gender: we will group "nan" and "unavailable" value to "unavailable" because it all provided us no further information about the object
- province_name: there is some mismatch labels, with full and shor-form of locations, in here I will choose and convert all into short-form labels of region for more proficiency,
- age: same mistakes with province, in here i will convert all into some range of age for effecttive storage 
- Beside that, based on data-leakage, I will assume that all values related to region will be treated as their most-appeared city in the table, like "Miền Bắc" will be defined as "Hanoi"

In [22]:
def gender_transform(x): 
    if pd.isna(x): 
        x = 'unavailable'
    return x

def province_name_transform(x): 
    if x in ('Miền Trung','ĐN','Đà Nẵng'): 
        x = 'mid_side'
    elif x in ('Hồ Chí Minh','HCM','Miền Nam'):
        x = 'south_side'
    elif x == 'UNIDENTIFIED':
        x = 'unavailable' 
    else: 
        x = 'north_side'
    return x 

def age_transform(x):
    desire_range = ['U18','18_24','25_34','35_44','45_54','O55']
    # U18 stands for under-18 and O55 stands for over-55
    if x in desire_range:
        return x
    elif  x == 'UNIDENTIFIED':
        x = 'unavailable' 
    elif  (x == 'Hai Tư') or (int(re.findall(r"(\d+)",str(x))[0]) <= 24): 
        x = '18_24'
    elif int(re.findall(r"(\d+)",str(x))[0]) < 18: 
        x = 'U18'
    elif int(re.findall(r"(\d+)",str(x))[0]) <= 34: 
        x = '25_34'
    elif int(re.findall(r"(\d+)",str(x))[0]) <= 44: 
        x = '35_44'
    elif int(re.findall(r"(\d+)",str(x))[0]) <= 54: 
        x = '45_54'
    else: 
        x = 'O55'
    return x

df_user['gender'] = df_user['gender'].apply(lambda x: gender_transform(x))
df_user['province_name'] = df_user['province_name'].apply(lambda x: province_name_transform(x))
df_user['age'] = df_user['age'].apply(lambda x: age_transform(x))

df_user.head()

,gender,user_id,province_name,age
0,F,76843,north_side,O55
1,unavailable,13346d5,mid_side,25_34
2,unavailable,132ed57,south_side,unavailable
3,F,ac6216,north_side,U18
4,M,cb2f6f,north_side,O55


In [23]:
def city_transform(x): 
    if x == 'north_side': 
        x = 'Hanoi'
    elif x == 'mid_side':
        x = 'Danang'
    elif x == 'south_side':
        x = 'HoChiMinhCity' 

    return x 

df_user['city'] = df_user['province_name'].apply(lambda x: city_transform(x))

df_user.head()

,gender,user_id,province_name,age,city
0,F,76843,north_side,O55,Hanoi
1,unavailable,13346d5,mid_side,25_34,Danang
2,unavailable,132ed57,south_side,unavailable,HoChiMinhCity
3,F,ac6216,north_side,U18,Hanoi
4,M,cb2f6f,north_side,O55,Hanoi


## 2.Table D_merchant

In [7]:
try:
    df_merchant = pd.read_excel(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_merchant')
except: 
    df_merchant = pd.read_excel(r'C:\Users\hoang\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_merchant')
    
df_merchant

,MERCHANT_CODE,category,sub_category
0,DOSALESFORCE22049Rbo,Nhà cửa - Đời sống,Dịch vụ hỗ trợ gia đình
1,bt9,Nhà cửa - Đời sống,Hoa & Quà lưu niệm
2,DOSALESFORCE23199Um5,Ẩm thực,Đồ uống
3,GOLDENGATE,Ẩm thực,Nhà hàng - Quán ăn
4,HIGHLANDS,Ẩm thực,Đồ uống
...,...,...,...
105,DOSALESFORCE24018VQL,Ẩm thực,Nhà hàng - Quán ăn
106,315798431FLOWERSTORE,Nhà cửa - Đời sống,Hoa & Quà lưu niệm
107,DOSALESFORCE240191s3,Ẩm thực,Đồ uống
108,DOSALESFORCE23058FeM,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ


In [8]:
validation_check(df_user)

Shape is: (66717, 4) 

Null values summary:


gender           0
user_id          0
province_name    0
age              0
dtype: int64


Types of data:


gender           object
user_id          object
province_name    object
age              object
dtype: object

In [9]:
print('Distinct values of each columns are:')
for col_name in ['category', 'sub_category']:
    print (f'{col_name}: {df_merchant[col_name].unique().tolist()} \n' ) 

Distinct values of each columns are:
category: ['Nhà cửa - Đời sống', 'Ẩm thực', 'Thời trang và Phong cách sống', 'Vận tải - Giao vận', 'Sức khỏe - Làm đẹp', 'Bán lẻ FMCG', 'Giáo dục', 'Khác', nan, 'Thương mại điện tử', 'Mẹ & bé', 'Dịch vụ tài chính', 'Tiện ích - Nhà mạng'] 

sub_category: ['Dịch vụ hỗ trợ gia đình', 'Hoa & Quà lưu niệm', 'Đồ uống', 'Nhà hàng - Quán ăn', 'Trang sức - Phụ kiện', 'Dịch vụ giao hàng', 'Spa - Thẩm mỹ', 'Cửa hàng tiện lợi', 'Gia dụng', 'Nha Khoa', 'Siêu thị', 'Hàng không', 'Bệnh viện - Phòng khám', 'Khóa học', 'Thời trang', 'Nội thất', 'Nha khoa', nan, 'Mỹ phẩm - Chăm sóc da, tóc', 'Phòng tập', 'Cửa hàng kem/bánh/kẹo', 'Thương mại điện tử', 'Bảo hiểm', 'Thực phẩm chức năng - CSSK', 'Ứng dụng gọi xe - Taxi', 'Vận tải - Giao vận khác', 'Viễn thông'] 



In [10]:
suspicious_merchant = df_merchant[df_merchant['sub_category'].isna()].MERCHANT_CODE.tolist()

display(df_merchant[df_merchant['sub_category'].isna()])

,MERCHANT_CODE,category,sub_category
23,0316282840MISHI,Khác,NaN
29,VinID,NaN,NaN
40,Innisfree,NaN,NaN
57,xXC,Mẹ & bé,NaN
66,xXC,Thương mại điện tử,NaN
73,DOSALESFORCE24088ABs,Mẹ & bé,NaN
94,DOSALESFORCE24036I0u,Mẹ & bé,NaN


Taking a closer look at the null values, we can see that null values occured at some merchant_code, which can be caused by:
- Uncategorize labels 
- Data Ingestion errors
- These merchant had unqiue category, which is not currently matching with any category and sub_category in system

So that, I will convert all this Nan_value into 'Uncategorized' for further investigation

In [11]:
df_merchant = df_merchant.fillna('Uncategorized')
df_merchant

,MERCHANT_CODE,category,sub_category
0,DOSALESFORCE22049Rbo,Nhà cửa - Đời sống,Dịch vụ hỗ trợ gia đình
1,bt9,Nhà cửa - Đời sống,Hoa & Quà lưu niệm
2,DOSALESFORCE23199Um5,Ẩm thực,Đồ uống
3,GOLDENGATE,Ẩm thực,Nhà hàng - Quán ăn
4,HIGHLANDS,Ẩm thực,Đồ uống
...,...,...,...
105,DOSALESFORCE24018VQL,Ẩm thực,Nhà hàng - Quán ăn
106,315798431FLOWERSTORE,Nhà cửa - Đời sống,Hoa & Quà lưu niệm
107,DOSALESFORCE240191s3,Ẩm thực,Đồ uống
108,DOSALESFORCE23058FeM,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ


In [35]:
counts = df_merchant['MERCHANT_CODE'].value_counts()

duplicates = counts[counts > 1]

print(duplicates)

MERCHANT_CODE
xXC                     11
VUAHANGHIEU              3
0316282840MISHI          2
DOSALESFORCE23359nRU     2
DOSALESFORCE22152Jzp     2
DOSALESFORCE23221RiK     2
Name: count, dtype: int64


In [26]:
df_merchant[df_merchant.MERCHANT_CODE == 'xXC']

,MERCHANT_CODE,category,sub_category
9,xXC,Bán lẻ FMCG,Cửa hàng tiện lợi
11,xXC,Vận tải - Giao vận,Dịch vụ giao hàng
13,xXC,Bán lẻ FMCG,Siêu thị
27,xXC,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
57,xXC,Mẹ & bé,Uncategorized
64,xXC,Ẩm thực,Nhà hàng - Quán ăn
66,xXC,Thương mại điện tử,Uncategorized
68,xXC,Ẩm thực,Đồ uống
76,xXC,Tiện ích - Nhà mạng,Viễn thông
86,xXC,Thời trang và Phong cách sống,Thời trang


In [151]:
df_merchant[df_merchant.MERCHANT_CODE.isin(duplicates.index)]

,MERCHANT_CODE,category,sub_category
9,xXC,Bán lẻ FMCG,Cửa hàng tiện lợi
11,xXC,Vận tải - Giao vận,Dịch vụ giao hàng
13,xXC,Bán lẻ FMCG,Siêu thị
18,DOSALESFORCE23221RiK,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ
23,0316282840MISHI,Khác,Uncategorized
27,xXC,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
36,DOSALESFORCE22152Jzp,Vận tải - Giao vận,Dịch vụ giao hàng
39,VUAHANGHIEU,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
57,xXC,Mẹ & bé,Uncategorized
61,DOSALESFORCE23359nRU,Sức khỏe - Làm đẹp,Thực phẩm chức năng - CSSK


You can see that we have some duplicates values of merchant in here, which can be caused by: 
- Merchant had multiple businesses in different category
- Wrong labels and coding (We supposed to have distinct Merchant_code for different category and sub_category) \


=> I will re match this in voucher, scheme and fact table below 

## 3.Table D_voucher

In [15]:
try:
    df_voucher = pd.read_excel(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_voucher')
except: 
    df_voucher = pd.read_excel(r'C:\Users\hoang\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_voucher')
    
df_voucher

,voucher_code,total_stock,display_date_from,display_date_to
0,MA_Hoayeuthuong_01032024_SF4672,400,2024-03-19 10:00:00,2024-04-30 23:59:00
1,MA_ROBINSTORE_19032024_SF4775,200,2024-03-29 10:00:00,2024-04-30 23:59:00
2,MAA_TUELAMBEAUTY_24012024_SF4562,500,2024-01-30 10:00:00,2024-03-31 23:59:00
3,MA_HIGHLANDS_01022024_SF4599,20000,2024-02-01 10:00:00,2024-02-29 23:59:00
4,MAP_CITiGYM_27062023_SF3464,9699,2023-06-27 10:00:00,2024-06-30 23:59:00
...,...,...,...,...
312,xXC10173,199,2023-09-26 10:00:00,2024-04-02 23:59:00
313,MA_KHRUABAANTHAI_18032024_SF4699,500,2024-03-25 10:00:00,2024-04-29 23:59:00
314,MA_CMSEDU_15112023_SF4204,500,2023-11-23 10:00:00,2024-01-01 23:59:00
315,MAA_TORANO_24102023_SF4114,200,2023-10-27 10:00:00,2024-06-30 21:00:00


In [16]:
validation_check(df_voucher)

Shape is: (317, 4) 

Null values summary:


voucher_code         0
total_stock          0
display_date_from    0
display_date_to      0
dtype: int64


Types of data:


voucher_code                 object
total_stock                   int64
display_date_from    datetime64[ns]
display_date_to      datetime64[ns]
dtype: object

Overall, there are no null value in gender dimensions. \
In here we will scan through 3 columns ['gender', 'province_name', 'age'] for any further in-valid value

In [17]:
df_voucher.shape

(317, 4)

In [18]:
df_voucher.isnull().sum()

voucher_code         0
total_stock          0
display_date_from    0
display_date_to      0
dtype: int64

In [19]:
df_voucher.dtypes

voucher_code                 object
total_stock                   int64
display_date_from    datetime64[ns]
display_date_to      datetime64[ns]
dtype: object

This table seems valid with acceptable format

## 4.Table D_scheme

In [20]:
try:
    df_scheme = pd.read_excel(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_scheme')
except: 
    df_scheme = pd.read_excel(r'C:\Users\hoang\OneDrive\Download\data DAE.xlsx',sheet_name = 'd_scheme')
    
df_scheme

,discount_type,discount_amount,discount_percent,description,VOUCHER_CODE
0,Cash Voucher,0,0,>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,xXC3723
1,Buy 2 get 1,0,0,>voucher ưu đãi >khi mua 02 ly trà cỡ lớn (áp ...,MA_HIGHLANDS_01022024_SF4601
2,Free SKU,0,0,>voucher miễn phí combo cắt + hấp tóc trị giá ...,MA_YTOHAIRSALON_02062023_SF4718
3,Discount Fixed Amount (For Total Bill),300000,0,">voucher giảm 300,000đ cho hóa đơn 1,500,000đ ...",MAA_TORANO_24102023_SF4114
4,Discount Percentage (For SKU),0,5,>voucher ưu đãi giảm 5% cho đơn hàng bất kỳ kh...,MAA_VUANEM_01022024_SF4596
...,...,...,...,...,...
310,Free SKU,0,0,">tặng kem nền double wear 7ml trị giá 380,000đ...",MA_ESTEELAUDER_05032024_SF4664
311,Discount Percentage (For SKU),0,43,">combo giảm 43% chỉ còn 465,000đ (giá gốc 820,...",MA_BONGON555_29032023_SF4748
312,Cash Voucher,50000,0,công của >highlands coffee> là: không gian qu...,xXC8525
313,Discount Percentage (For SKU),0,30,>giảm 30% gói pinkcare - xét nghiệm tầm soát n...,MAA_GENSOLUTION_25122023_SF4416


In [21]:
validation_check(df_scheme)

Shape is: (315, 5) 

Null values summary:


discount_type       0
discount_amount     0
discount_percent    0
description         0
VOUCHER_CODE        0
dtype: int64


Types of data:


discount_type       object
discount_amount      int64
discount_percent     int64
description         object
VOUCHER_CODE        object
dtype: object

Overall, there are no null value in gender dimensions. \
In here we will scan through 3 columns ['gender', 'province_name', 'age'] for any further in-valid value

## 5.Table Fact

In [22]:
try:
    df_fact = pd.read_excel(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\data DAE.xlsx',sheet_name = 'fact')
except: 
    df_fact = pd.read_excel(r'C:\Users\hoang\OneDrive\Download\data DAE.xlsx',sheet_name = 'fact')
    
df_fact = df_fact.sort_values(by = ['USER_ID','CALENDAR_DIM_ID']).reset_index(drop = True)

df_fact

,TRANSACTION_ID,SERIAL_NUMBER,VOUCHER_CODE,VOUCHER_NAME,AVAILABLE_FROM,AVAILABLE_TO,MERCHANT_CODE,ACTION,CALENDAR_DIM_ID,USER_ID
0,136701,VIN03Y6ZV2H,MA_BTASKEE_02062023_SF3423,Tặng 01 giờ Dịch vụ giúp việc tại nhà,2023-06-02T10:00,2024-02-29T23:59,DOSALESFORCE22049Rbo,Claimed,2024-01-19T00:00,10009
1,389269,VIBTK64MKY8U,MA_BTASKEE_02062023_SF3424,"Giảm 30,000đ Dịch vụ dọn dẹp nhà cửa",2023-06-02T10:00,2024-02-29T23:59,DOSALESFORCE22049Rbo,Claimed,2024-01-19T00:00,10009
2,31758,59IS50TR,MAA_KFC_01012024_SF4346,"Giảm 30,000đ cho hóa đơn từ 130,000đ",2024-01-01T10:00,2024-03-31T23:59,DOSALESFORCE23229Twc,Claimed,2024-02-14T00:00,1001
3,249756,CVLHC4Z1,MAA_KFC_01012024_SF4346,"Giảm 30,000đ cho hóa đơn từ 130,000đ",2024-01-01T10:00,2024-03-31T23:59,DOSALESFORCE23229Twc,Claimed,2024-02-14T00:00,1001
4,363195,N0MS91SE,MAA_KFC_01012024_SF4346,"Giảm 30,000đ cho hóa đơn từ 130,000đ",2024-01-01T10:00,2024-03-31T23:59,DOSALESFORCE23229Twc,Claimed,2024-02-14T00:00,1001
...,...,...,...,...,...,...,...,...,...,...
303938,311355,823009728434287,xXC3706,"Thẻ nạp Viettel 30,000đ",2023-11-01T15:00,2024-12-31T23:59,xXC,Claimed,2024-02-12T00:00,fff9d
303939,11109,BCVINIDYGQEN40Z,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,2024-01-18T10:00,2024-06-30T23:59,DOSALESFORCE22152Jzp,Claimed,2024-02-26T00:00,fffdb
303940,21330,BCVINIDYGUERWE6,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,2024-01-18T10:00,2024-06-30T23:59,DOSALESFORCE22152Jzp,Claimed,2024-02-26T00:00,fffdb
303941,293058,BCVINIDYGPJRRNW,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,2024-01-18T10:00,2024-06-30T23:59,DOSALESFORCE22152Jzp,Claimed,2024-02-26T00:00,fffdb


In [173]:
df_scheme

,discount_type,discount_amount,discount_percent,description,VOUCHER_CODE
0,Cash Voucher,0,0,>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,xXC3723
1,Buy 2 get 1,0,0,>voucher ưu đãi >khi mua 02 ly trà cỡ lớn (áp ...,MA_HIGHLANDS_01022024_SF4601
2,Free SKU,0,0,>voucher miễn phí combo cắt + hấp tóc trị giá ...,MA_YTOHAIRSALON_02062023_SF4718
3,Discount Fixed Amount (For Total Bill),300000,0,">voucher giảm 300,000đ cho hóa đơn 1,500,000đ ...",MAA_TORANO_24102023_SF4114
4,Discount Percentage (For SKU),0,5,>voucher ưu đãi giảm 5% cho đơn hàng bất kỳ kh...,MAA_VUANEM_01022024_SF4596
...,...,...,...,...,...
310,Free SKU,0,0,">tặng kem nền double wear 7ml trị giá 380,000đ...",MA_ESTEELAUDER_05032024_SF4664
311,Discount Percentage (For SKU),0,43,">combo giảm 43% chỉ còn 465,000đ (giá gốc 820,...",MA_BONGON555_29032023_SF4748
312,Cash Voucher,50000,0,công của >highlands coffee> là: không gian qu...,xXC8525
313,Discount Percentage (For SKU),0,30,>giảm 30% gói pinkcare - xét nghiệm tầm soát n...,MAA_GENSOLUTION_25122023_SF4416


As I stated above, we have some errors in merchant, fact and scheme table, which can't match the category and subcategory of it's vouchers, in here I will state some solutions for this problems, such as: 

- Hand labeling each based on their attributes (voucher name and description), which will time wasting and burden because it's almost 100 unlabeled voucher 
- Build some text-based algorithm like neural-nets for classification, which will help us solve the long-term problems whenver we have a mismatch ones , but this solution can be costly hand not available due to tight-deadline
- Using some pretrained model or some GPT api for classification => This solution will be perfectly fit due to its flexibility and cheap, also very easy to implement

Also, for use, I will only classify / label voucher which had been delivered to user (the one appeared in df_fact table instead of labeling all appeared vouchers)

In [172]:
dup_merchant_fact = df_fact.loc[df_fact['MERCHANT_CODE'].isin(duplicates.index),['MERCHANT_CODE','VOUCHER_CODE','VOUCHER_NAME']].drop_duplicates().reset_index(drop = True)
dup_merchant_scheme = df_scheme.loc[df_scheme['VOUCHER_CODE'].isin(dup_merchant_fact['VOUCHER_CODE']),['VOUCHER_CODE','description']].drop_duplicates().reset_index(drop = True)
merchant_cate = df_merchant.loc[df_merchant['MERCHANT_CODE'].isin(duplicates.index)].reset_index(drop = True)


test = dup_merchant_fact.merge(dup_merchant_scheme, on = ['VOUCHER_CODE'])

co = cohere.Client("INSERT_YOUR_SECRET_KEY")

def get_gpt_response(prompt):
    response = co.chat(
        message= prompt
    )

    time.sleep(3)
    return response.text


test['result'] = test.apply(lambda x: get_gpt_response(
    f""" Giả sử tôi có 2 thông tin như sau cho 1 mã giảm giá:

        {x['VOUCHER_NAME']} ,

        {x['description']}
        
        Thêm vào đó merchant_code của mã này là x['MERCHANT_CODE']
        
        Dựa vào cả 2 thông tin thông tin về text như trên, lựa chọn giúp tôi category đi kèm với sub_category hợp lý nhất cho mã giảm giá này theo như bảng bên dưới
        Không cần phải giải thích, Định dạng tôi cần chỉ là 2 phần category ,  sub_category, cách nhau bởi dấu ","
        
        {merchant_cate}

    """
    ), axis = 1
)

test

,MERCHANT_CODE,VOUCHER_CODE,VOUCHER_NAME,description,result
0,xXC,xXC2654,"Mã nạp Tiki Xu 20,000đ",">mã nạp tiki xu trị giá 20,000đ tương đương vớ...","Thương mại điện tử, Uncategorized"
1,DOSALESFORCE22152Jzp,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,">voucher giảm 20% tối đa 50,000đ cho chuyến đi...","Vận tải - Giao vận, Dịch vụ giao hàng"
2,xXC,xXC3723,"Thẻ nạp Mobifone 20,000đ",>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,"Thương mại điện tử, Viễn thông"
3,xXC,xXC12555,"Voucher 20,000đ Mua hàng tiện lợi",">voucher trị giá 20,000đ> áp dụng khi mua hàng...","Bán lẻ FMCG, Cửa hàng tiện lợi"
4,xXC,xXC9242,"Voucher 20,000đ Mỹ phẩm làm đẹp xu hướng",ở thành phiên bản hoàn hảo hơn mỗi ngày.,"Sức khỏe - Làm đẹp, Mỹ phẩm - Chăm sóc da, tóc"
...,...,...,...,...,...
81,0316282840MISHI,MA_KACHI_18102023_SF4032,Giảm 45% Bàn chải điện Kachi Sonic MK309,"hoạt động, đảm bảo thời gian đánh răng tiêu ch...","Sức khỏe - Làm đẹp, Mỹ phẩm - Chăm sóc da, tóc"
82,VUAHANGHIEU,MA_LACOSTE_16112023_SF4211,"Giảm 80,000đ Giày da cao cấp",">voucher ưu đãi giảm 80,000đ khi mua giày >lac...","Thời trang và Phong cách sống, Giày dép"
83,xXC,xXC11198,"Voucher 50,000đ Vương quốc đồ chơi","ư đồ nghề ảo thuật, dụng cụ nhà bếp, đồ dùng l...","Bán lẻ FMCG, Cửa hàng tiện lợi"
84,xXC,xXC12558,"Voucher 50,000đ Mua hàng tiện lợi",">voucher trị giá 50,000đ> áp dụng khi mua hàng...","Bán lẻ FMCG, Cửa hàng tiện lợi"


In [194]:
test[['category','sub_category']] = test['result'].str.split(r',\s*', n=1, expand=True)

test = test.drop(columns = 'result')

test

,MERCHANT_CODE,VOUCHER_CODE,VOUCHER_NAME,description,category,sub_category
0,xXC,xXC2654,"Mã nạp Tiki Xu 20,000đ",">mã nạp tiki xu trị giá 20,000đ tương đương vớ...",Thương mại điện tử,Uncategorized
1,DOSALESFORCE22152Jzp,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,">voucher giảm 20% tối đa 50,000đ cho chuyến đi...",Vận tải - Giao vận,Dịch vụ giao hàng
2,xXC,xXC3723,"Thẻ nạp Mobifone 20,000đ",>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,Thương mại điện tử,Viễn thông
3,xXC,xXC12555,"Voucher 20,000đ Mua hàng tiện lợi",">voucher trị giá 20,000đ> áp dụng khi mua hàng...",Bán lẻ FMCG,Cửa hàng tiện lợi
4,xXC,xXC9242,"Voucher 20,000đ Mỹ phẩm làm đẹp xu hướng",ở thành phiên bản hoàn hảo hơn mỗi ngày.,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
...,...,...,...,...,...,...
81,0316282840MISHI,MA_KACHI_18102023_SF4032,Giảm 45% Bàn chải điện Kachi Sonic MK309,"hoạt động, đảm bảo thời gian đánh răng tiêu ch...",Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
82,VUAHANGHIEU,MA_LACOSTE_16112023_SF4211,"Giảm 80,000đ Giày da cao cấp",">voucher ưu đãi giảm 80,000đ khi mua giày >lac...",Thời trang và Phong cách sống,Giày dép
83,xXC,xXC11198,"Voucher 50,000đ Vương quốc đồ chơi","ư đồ nghề ảo thuật, dụng cụ nhà bếp, đồ dùng l...",Bán lẻ FMCG,Cửa hàng tiện lợi
84,xXC,xXC12558,"Voucher 50,000đ Mua hàng tiện lợi",">voucher trị giá 50,000đ> áp dụng khi mua hàng...",Bán lẻ FMCG,Cửa hàng tiện lợi


In [185]:
non_dup_merchant_fact = df_fact.loc[~df_fact['MERCHANT_CODE'].isin(duplicates.index),['MERCHANT_CODE','VOUCHER_CODE','VOUCHER_NAME']].drop_duplicates().reset_index(drop = True)
non_dup_merchant_scheme = df_scheme.loc[df_scheme['VOUCHER_CODE'].isin(non_dup_merchant_fact['VOUCHER_CODE']),['VOUCHER_CODE','description']].drop_duplicates().reset_index(drop = True)
non_dup_merchant = df_merchant.loc[df_merchant['MERCHANT_CODE'].isin(non_dup_merchant_fact['MERCHANT_CODE'])].drop_duplicates().reset_index(drop = True)

non_test = non_dup_merchant_fact.merge(non_dup_merchant_scheme, on = ['VOUCHER_CODE'], how = 'left')
non_test = non_test.merge(non_dup_merchant, on = ['MERCHANT_CODE'], how = 'left')

non_test

,MERCHANT_CODE,VOUCHER_CODE,VOUCHER_NAME,description,category,sub_category
0,DOSALESFORCE22049Rbo,MA_BTASKEE_02062023_SF3423,Tặng 01 giờ Dịch vụ giúp việc tại nhà,>voucher ưu đãi tặng 01 giờ dịch vụ dọn nhà >t...,Nhà cửa - Đời sống,Dịch vụ hỗ trợ gia đình
1,DOSALESFORCE22049Rbo,MA_BTASKEE_02062023_SF3424,"Giảm 30,000đ Dịch vụ dọn dẹp nhà cửa",">giảm ngay 30,000đ áp dụng cho tất cả dịch vụ ...",Nhà cửa - Đời sống,Dịch vụ hỗ trợ gia đình
2,DOSALESFORCE23229Twc,MAA_KFC_01012024_SF4346,"Giảm 30,000đ cho hóa đơn từ 130,000đ",">ưu đãi giảm 30,000đ cho hóa đơn từ 130,000đ t...",Ẩm thực,Nhà hàng - Quán ăn
3,HIGHLANDS,MA_HIGHLANDS_02012024_SF4406,"Giảm 29,000đ cho hóa đơn từ 119,000đ",">voucher ưu đãi giảm 29,000đ cho hóa đơn từ 11...",Ẩm thực,Đồ uống
4,HIGHLANDS,MA_HIGHLANDS_02012024_SF4414,Mua 02 tặng 01 Trà và Freeze,>voucher ưu đãi >khi mua 02 ly trà hoặc freeze...,Ẩm thực,Đồ uống
...,...,...,...,...,...,...
175,DOSALESFORCE23058FeM,MAA_YENTRANGBEAUTY_24012024_SF4578,Giảm 25% Cấy vi điểm tăng sinh collagen,>- phục hồi sự đầy đặn cho khuôn mặt mang đến ...,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ
176,MOID20240gkf,MA_CMSEDU_15112023_SF4204,Học thử và kiểm tra năng lực tư duy,>ưu đãi tặng buổi kiểm tra năng lực tư duy + 0...,Giáo dục,Khóa học
177,DOSALESFORCE223418Dq,MA_TMVNGOCDUNG_12102023_SF3958,Tặng 01 suất Vàng trẻ hóa tái tạo da,collagen diễn ra liên tục.,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ
178,DOSALESFORCE24030P44,MAA_VENESA_22012024_SF4606,"Giảm 1,000,000đ Nâng cơ và Trẻ hóa da","hi sử dụng dịch vụ ưu đãi này, khách hàng được...",Sức khỏe - Làm đẹp,Spa - Thẩm mỹ


In [196]:
df_merchant_sum_up = pd.concat([test,non_test])
df_merchant_sum_up

,MERCHANT_CODE,VOUCHER_CODE,VOUCHER_NAME,description,category,sub_category
0,xXC,xXC2654,"Mã nạp Tiki Xu 20,000đ",">mã nạp tiki xu trị giá 20,000đ tương đương vớ...",Thương mại điện tử,Uncategorized
1,DOSALESFORCE22152Jzp,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,">voucher giảm 20% tối đa 50,000đ cho chuyến đi...",Vận tải - Giao vận,Dịch vụ giao hàng
2,xXC,xXC3723,"Thẻ nạp Mobifone 20,000đ",>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,Thương mại điện tử,Viễn thông
3,xXC,xXC12555,"Voucher 20,000đ Mua hàng tiện lợi",">voucher trị giá 20,000đ> áp dụng khi mua hàng...",Bán lẻ FMCG,Cửa hàng tiện lợi
4,xXC,xXC9242,"Voucher 20,000đ Mỹ phẩm làm đẹp xu hướng",ở thành phiên bản hoàn hảo hơn mỗi ngày.,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"
...,...,...,...,...,...,...
175,DOSALESFORCE23058FeM,MAA_YENTRANGBEAUTY_24012024_SF4578,Giảm 25% Cấy vi điểm tăng sinh collagen,>- phục hồi sự đầy đặn cho khuôn mặt mang đến ...,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ
176,MOID20240gkf,MA_CMSEDU_15112023_SF4204,Học thử và kiểm tra năng lực tư duy,>ưu đãi tặng buổi kiểm tra năng lực tư duy + 0...,Giáo dục,Khóa học
177,DOSALESFORCE223418Dq,MA_TMVNGOCDUNG_12102023_SF3958,Tặng 01 suất Vàng trẻ hóa tái tạo da,collagen diễn ra liên tục.,Sức khỏe - Làm đẹp,Spa - Thẩm mỹ
178,DOSALESFORCE24030P44,MAA_VENESA_22012024_SF4606,"Giảm 1,000,000đ Nâng cơ và Trẻ hóa da","hi sử dụng dịch vụ ưu đãi này, khách hàng được...",Sức khỏe - Làm đẹp,Spa - Thẩm mỹ


In [186]:
non_test.isna().sum()

MERCHANT_CODE    0
VOUCHER_CODE     0
VOUCHER_NAME     0
description      2
category         0
sub_category     0
dtype: int64

In [27]:
# Save all csv and ingest to powerBi for dashboard

# df_user.to_csv(r'C:\Users\hoang\OneDrive\Download\df_user.csv',index=False)
# df_merchant.to_csv(r'C:\Users\hoang\OneDrive\Download\df_merchant.csv',index=False)
# df_voucher.to_csv(r'C:\Users\hoang\OneDrive\Download\df_voucher.csv',index=False)
# df_scheme.to_csv(r'C:\Users\hoang\OneDrive\Download\df_scheme.csv',index=False)
# df_fact.to_csv(r'C:\Users\hoang\OneDrive\Download\df_fact.csv',index=False)
# df_merchant_sum_up.to_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_merchant_sum_up.csv',index=False)

# Analyzing

In [5]:
try:
    df_fact = pd.read_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_fact.csv')
except: 
    df_fact = pd.read_csv(r'C:\Users\hoang\OneDrive\Download\df_fact.csv')

df_fact['CALENDAR_DIM_ID'] = pd.to_datetime(df_fact['CALENDAR_DIM_ID'])

df_fact.head(5)

,TRANSACTION_ID,SERIAL_NUMBER,VOUCHER_CODE,VOUCHER_NAME,AVAILABLE_FROM,AVAILABLE_TO,MERCHANT_CODE,ACTION,CALENDAR_DIM_ID,USER_ID
0,1,2303368734,MA_HIGHLANDS_02012024_SF4410,"Giảm 39,000đ cho hóa đơn từ 139,000đ",2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Claimed,2024-01-02,12fcb2
1,2,9.16737E+14,xXC3707,"Thẻ nạp Viettel 50,000đ",2020-09-14T14:35,2024-12-31T23:59,xXC,Redeemed,2024-02-02,c9877a
2,3,2283370694,MA_HIGHLANDS_02012024_SF4410,"Giảm 39,000đ cho hóa đơn từ 139,000đ",2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Redeemed,2024-01-05,b1ebc3
3,5,2194823027,MA_HIGHLANDS_01022024_SF4602,"Giảm 39,000đ cho hóa đơn từ 149,000đ",2024-02-01T10:00,2024-02-29T23:59,HIGHLANDS,Claimed,2024-02-20,3e3bb8
4,6,VIN03IA6GQM,MA_BTASKEE_02062023_SF3423,Tặng 01 giờ Dịch vụ giúp việc tại nhà,2023-06-02T10:00,2024-02-29T23:59,DOSALESFORCE22049Rbo,Claimed,2024-02-10,a8ee0c


In [6]:
try:
    df_merchant_sum_up = pd.read_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_merchant_sum_up.csv')
except: 
    df_merchant_sum_up = pd.read_csv(r'C:\Users\hoang\OneDrive\Download\df_merchant_sum_up.csv')
    
df_merchant_sum_up.head(5)

,MERCHANT_CODE,VOUCHER_CODE,VOUCHER_NAME,description,category,sub_category
0,xXC,xXC2654,"Mã nạp Tiki Xu 20,000đ",">mã nạp tiki xu trị giá 20,000đ tương đương vớ...",Thương mại điện tử,Uncategorized
1,DOSALESFORCE22152Jzp,MA_BE_10012024_SF4486,Giảm 20% Di chuyển bằng beBike/beCar,">voucher giảm 20% tối đa 50,000đ cho chuyến đi...",Vận tải - Giao vận,Dịch vụ giao hàng
2,xXC,xXC3723,"Thẻ nạp Mobifone 20,000đ",>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,Thương mại điện tử,Viễn thông
3,xXC,xXC12555,"Voucher 20,000đ Mua hàng tiện lợi",">voucher trị giá 20,000đ> áp dụng khi mua hàng...",Bán lẻ FMCG,Cửa hàng tiện lợi
4,xXC,xXC9242,"Voucher 20,000đ Mỹ phẩm làm đẹp xu hướng",ở thành phiên bản hoàn hảo hơn mỗi ngày.,Sức khỏe - Làm đẹp,"Mỹ phẩm - Chăm sóc da, tóc"


In [21]:
try:
    df_user = pd.read_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_user.csv')
except: 
    df_user = pd.read_csv(r'C:\Users\hoang\OneDrive\Download\df_user.csv')
    
df_user.head(5)

,gender,user_id,province_name,age,city
0,F,76843,north_side,O55,Hanoi
1,unavailable,13346d5,mid_side,25_34,Danang
2,unavailable,132ed57,south_side,unavailable,HoChiMinhCity
3,F,ac6216,north_side,U18,Hanoi
4,M,cb2f6f,north_side,O55,Hanoi


In [7]:
try:
    df_scheme = pd.read_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_scheme.csv')
except: 
    df_scheme = pd.read_csv(r'C:\Users\hoang\OneDrive\Download\df_scheme.csv')

df_scheme.head(5)

,discount_type,discount_amount,discount_percent,description,VOUCHER_CODE
0,Cash Voucher,0,0,>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,xXC3723
1,Buy 2 get 1,0,0,>voucher ưu đãi >khi mua 02 ly trà cỡ lớn (áp ...,MA_HIGHLANDS_01022024_SF4601
2,Free SKU,0,0,>voucher miễn phí combo cắt + hấp tóc trị giá ...,MA_YTOHAIRSALON_02062023_SF4718
3,Discount Fixed Amount (For Total Bill),300000,0,">voucher giảm 300,000đ cho hóa đơn 1,500,000đ ...",MAA_TORANO_24102023_SF4114
4,Discount Percentage (For SKU),0,5,>voucher ưu đãi giảm 5% cho đơn hàng bất kỳ kh...,MAA_VUANEM_01022024_SF4596


In [16]:
try:
    df_voucher = pd.read_csv(r'C:\Users\CPU11988_LOCAL\OneDrive\Download\df_voucher.csv')
except: 
    df_voucher = pd.read_csv(r'C:\Users\hoang\OneDrive\Download\df_voucher.csv')
df_voucher['display_date_from'] = pd.to_datetime(df_voucher['display_date_from'])
df_voucher['display_date_to'] = pd.to_datetime(df_voucher['display_date_to'])

df_voucher.head(5)

,voucher_code,total_stock,display_date_from,display_date_to
0,MA_Hoayeuthuong_01032024_SF4672,400,2024-03-19 10:00:00,2024-04-30 23:59:00
1,MA_ROBINSTORE_19032024_SF4775,200,2024-03-29 10:00:00,2024-04-30 23:59:00
2,MAA_TUELAMBEAUTY_24012024_SF4562,500,2024-01-30 10:00:00,2024-03-31 23:59:00
3,MA_HIGHLANDS_01022024_SF4599,20000,2024-02-01 10:00:00,2024-02-29 23:59:00
4,MAP_CITiGYM_27062023_SF3464,9699,2023-06-27 10:00:00,2024-06-30 23:59:00


In [25]:
final = df_fact.merge(df_merchant_sum_up.loc[:,['VOUCHER_CODE','description','category','sub_category']], on = ['VOUCHER_CODE'], how = 'left')
final = final.merge(df_scheme.loc[:,['discount_type','discount_amount','discount_percent','VOUCHER_CODE']], on = ['VOUCHER_CODE'], how = 'left')
final = final.merge(df_user, left_on = ['USER_ID'], right_on = ['user_id']).drop(columns = ['user_id'])
final.head(5)

,TRANSACTION_ID,SERIAL_NUMBER,VOUCHER_CODE,VOUCHER_NAME,AVAILABLE_FROM,AVAILABLE_TO,MERCHANT_CODE,ACTION,CALENDAR_DIM_ID,USER_ID,description,category,sub_category,discount_type,discount_amount,discount_percent,gender,province_name,age,city
0,1,2303368734,MA_HIGHLANDS_02012024_SF4410,"Giảm 39,000đ cho hóa đơn từ 139,000đ",2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Claimed,2024-01-02,12fcb2,">voucher ưu đãi giảm 39,000đ cho hóa đơn từ 13...",Ẩm thực,Đồ uống,Discount Fixed Amount (For Total Bill),39000.0,0.0,M,south_side,18_24,HoChiMinhCity
1,1827,2127020034,MA_HIGHLANDS_02012024_SF4414,Mua 02 tặng 01 Trà và Freeze,2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Claimed,2024-01-02,12fcb2,>voucher ưu đãi >khi mua 02 ly trà hoặc freeze...,Ẩm thực,Đồ uống,Buy 2 get 1,0.0,0.0,M,south_side,18_24,HoChiMinhCity
2,2053,2304268734,MA_HIGHLANDS_02012024_SF4406,"Giảm 29,000đ cho hóa đơn từ 119,000đ",2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Claimed,2024-01-02,12fcb2,">voucher ưu đãi giảm 29,000đ cho hóa đơn từ 11...",Ẩm thực,Đồ uống,Discount Fixed Amount (For Total Bill),29000.0,0.0,M,south_side,18_24,HoChiMinhCity
3,2299,2230843537,MA_HIGHLANDS_01022024_SF4600,Tặng 01 ly Trà size S,2024-02-01T10:00,2024-02-29T23:59,HIGHLANDS,Claimed,2024-02-01,12fcb2,>voucher ưu đãi tặng 01 ly trà size s (trị giá...,Ẩm thực,Đồ uống,Free SKU,0.0,0.0,M,south_side,18_24,HoChiMinhCity
4,3273,2293368724,MA_HIGHLANDS_02012024_SF4410,"Giảm 39,000đ cho hóa đơn từ 139,000đ",2024-01-02T10:00,2024-01-31T23:59,HIGHLANDS,Claimed,2024-01-02,12fcb2,">voucher ưu đãi giảm 39,000đ cho hóa đơn từ 13...",Ẩm thực,Đồ uống,Discount Fixed Amount (For Total Bill),39000.0,0.0,M,south_side,18_24,HoChiMinhCity


In [11]:
sum_up = final.groupby('ACTION', as_index = False).agg(
    users = ('USER_ID', lambda x: len(set(x))),
    __times = ('TRANSACTION_ID', lambda x: len(set(x)))
)

sum_up['avg_times_per_users'] = round(sum_up['__times'] / sum_up['users'],2)

sum_up

,ACTION,users,__times,avg_times_per_users
0,Claimed,50421,209345,4.15
1,Redeemed,33275,94598,2.84


In [14]:
summary_discount = final[final['ACTION'] == 'Redeemed'].groupby('discount_type').agg(
    total_value = ('discount_amount',lambda x: f"{sum(x):,.2f}"),
    avg_value = ('discount_amount',lambda x: f"{np.mean(x):,.2f}"), 
    redeemed_vouchers = ('SERIAL_NUMBER', lambda x: len(set(x)))
).sort_values(by='redeemed_vouchers', ascending=False)

summary_discount['stock'] = df_voucher.merge(final.loc[:,['discount_type','VOUCHER_CODE']].drop_duplicates(), 
                 how = 'left', 
                 left_on = ['voucher_code'], 
                 right_on = ['VOUCHER_CODE']).groupby('discount_type')['total_stock'].sum()

summary_discount['redeemed_rate'] = round(summary_discount['redeemed_vouchers'] / summary_discount['stock'],2)

summary_discount

,total_value,avg_value,redeemed_vouchers,stock,redeemed_rate
discount_type,,,,,
Cash Voucher,"77,521,000.00","2,102.26",35358,4907705,0.01
Discount Fixed Amount (For Total Bill),"1,344,427,000.00","38,552.09",34870,164119,0.21
Discount Percentage (For SKU),"11,443,200.00",811.11,14108,124584,0.11
Buy 2 get 1,0.00,0.00,4057,72948,0.06
Free SKU,0.00,0.00,2641,183474,0.01
Discount Fixed Amount (For SKU),"125,740,000.00","88,736.77",1417,82406,0.02
Discount Percentage (For Total Bill),0.00,0.00,477,29452,0.02
Buy 1 get 1,0.00,0.00,92,1100,0.08
Others,0.00,0.00,17,28171,0.00


In [17]:
df_voucher[
    (df_voucher['display_date_to'] < pd.to_datetime('2024-02-14')) &
    (df_voucher['display_date_to'] > pd.to_datetime('2024-02-07'))
]

,voucher_code,total_stock,display_date_from,display_date_to
262,MAA_SAPPACADEMY_30122023_SF4423,20,2024-01-03 10:00:00,2024-02-08 23:59:00
311,xXC12558,40,2023-11-08 14:00:00,2024-02-07 23:59:00


In [18]:
down_trend = final[
    (final['CALENDAR_DIM_ID'] >= pd.to_datetime('2024-02-04')) & 
    (final['CALENDAR_DIM_ID'] < pd.to_datetime('2024-02-14'))
].groupby(['VOUCHER_CODE','CALENDAR_DIM_ID','category','sub_category'],as_index = False).agg(
    redeemed_vouchers = ('SERIAL_NUMBER',lambda x: len(set(x)))
)


down_trend.pivot(columns= 'CALENDAR_DIM_ID', index= ['VOUCHER_CODE','category','sub_category'], values= 'redeemed_vouchers').fillna(0).sort_values(by = "2024-02-06", ascending= False).head(15)

,,CALENDAR_DIM_ID,2024-02-04,2024-02-05,2024-02-06,2024-02-07,2024-02-08,2024-02-09,2024-02-10,2024-02-11,2024-02-12,2024-02-13
VOUCHER_CODE,category,sub_category,,,,,,,,,,
MA_HIGHLANDS_01022024_SF4602,Ẩm thực,Đồ uống,931.0,825.0,804.0,801.0,498.0,347.0,419.0,452.0,585.0,500.0
MA_HIGHLANDS_01022024_SF4599,Ẩm thực,Đồ uống,812.0,692.0,610.0,576.0,417.0,231.0,242.0,254.0,345.0,351.0
xXC3707,Thương mại điện tử,Viễn thông,240.0,265.0,372.0,588.0,334.0,264.0,130.0,120.0,119.0,181.0
MA_PNJ_12012024_SF4479,Thời trang và Phong cách sống,Trang sức - Phụ kiện,238.0,236.0,357.0,285.0,238.0,49.0,39.0,51.0,80.0,228.0
MA_PNJ_12012024_SF4480,Thời trang và Phong cách sống,Trang sức - Phụ kiện,232.0,260.0,346.0,410.0,245.0,16.0,7.0,25.0,25.0,232.0
MA_HIGHLANDS_01022024_SF4601,Ẩm thực,Đồ uống,369.0,347.0,297.0,303.0,206.0,140.0,185.0,195.0,243.0,243.0
MAA_KFC_01012024_SF4346,Ẩm thực,Nhà hàng - Quán ăn,261.0,201.0,280.0,236.0,211.0,94.0,142.0,177.0,209.0,246.0
MA_HIGHLANDS_01022024_SF4600,Ẩm thực,Đồ uống,254.0,228.0,198.0,213.0,143.0,61.0,110.0,97.0,133.0,147.0
MA_PNJSILVER_12012024_SF4478,Thời trang và Phong cách sống,Trang sức - Phụ kiện,83.0,107.0,174.0,113.0,105.0,9.0,2.0,8.0,16.0,83.0


In [22]:
df_user

,gender,user_id,province_name,age,city
0,F,76843,north_side,O55,Hanoi
1,unavailable,13346d5,mid_side,25_34,Danang
2,unavailable,132ed57,south_side,unavailable,HoChiMinhCity
3,F,ac6216,north_side,U18,Hanoi
4,M,cb2f6f,north_side,O55,Hanoi
...,...,...,...,...,...
66712,F,c91280,south_side,35_44,HoChiMinhCity
66713,F,12fa13,mid_side,18_24,Danang
66714,F,1022ae,south_side,35_44,HoChiMinhCity
66715,M,119452,south_side,35_44,HoChiMinhCity


In [66]:
pd.options.display.float_format = '${:,.2f}'.format

demographic = final[final['ACTION'] == 'Redeemed'].fillna(0).groupby(['province_name','age'],as_index = False).agg(
    redeemed_times = ('discount_amount',lambda x: len(x)),
    redeemed_users = ('USER_ID',lambda x: len(set(x))),
    value = ('discount_amount',lambda x: sum(x)),
    percent = ('discount_percent',lambda x: sum(x) / len(x))
)

demographic['value_per_user'] = demographic['value'] / demographic['redeemed_users']

demographic.pivot(index = 'age', columns = 'province_name')[['value','value_per_user']]

value                                                 
province_name        mid_side      north_side     south_side    unavailable   
age                                                                           
18_24          $45,187,000.00  $60,920,000.00 $50,436,000.00 $13,142,000.00  \
25_34         $105,669,400.00 $129,463,000.00 $92,098,000.00 $30,771,000.00   
35_44          $86,574,000.00 $113,417,000.00 $78,184,000.00 $26,124,000.00   
45_54          $51,217,400.00  $78,561,000.00 $61,146,000.00 $15,407,000.00   
O55            $44,857,000.00  $66,557,000.00 $44,912,400.00 $14,030,000.00   
U18            $41,807,000.00  $62,311,000.00 $48,118,000.00 $13,817,000.00   
unavailable    $39,674,000.00  $56,043,000.00 $43,262,000.00 $15,924,000.00   

              value_per_user                                    
province_name       mid_side north_side south_side unavailable  
age                                                             
18_24             $44,783.94 $48,234.36 $47,357.75  $41,327.04  
25_34             $53,153.62 $49,150.72 $46,940.88  $47,122.51  
35_44             $48,582.49 $47,415.13 $44,625.57  $47,671.53  
45_54             $40,713.35 $48,404.81 $49,271.56  $37,486.62  
O55               $45,037.15 $49,264.99 $45,735.64  $45,404.53  
U18               $42,923.00 $46,956.29 $51,026.51  $44,427.65  
unavailable       $40,816.87 $42,456.82 $44,462.49  $48,996.92

In [84]:
df_scheme.loc[3,'description']

'>voucher giảm 300,000đ cho hóa đơn 1,500,000đ áp dụng với hàng nguyên giá tại thời trang torano.>giá'

In [90]:
co = cohere.Client("INSERT_YOUR_SECRET_KEY")

def get_gpt_response(prompt):
    response = co.chat(
        message= prompt
    )

    time.sleep(3)
    return response.text


df_scheme['minimum_bill'] = df_scheme['description'].apply(lambda x: get_gpt_response(
    f""" 
    Dựa vào thông tin dưới:

    {x}

    Hãy cho tôi biết điều kiện về giá trị tối thiểu của voucher này, nếu không có yêu cầu thì thì ghi 0. Chỉ cần ghi kết quả không cần giải thích

    """
    )
)

df_scheme['minimum_bill'] = df_scheme['minimum_bill'].apply(lambda x: x.replace(",", ""))
df_scheme['minimum_bill'] = df_scheme['minimum_bill'].apply(lambda x: 20000 if x == '20000 VND' else x)
df_scheme['minimum_bill'] = df_scheme['minimum_bill'].astype(int)

df_scheme

,discount_type,discount_amount,discount_percent,description,VOUCHER_CODE,minimum_bill
0,Cash Voucher,0,0,>đổi điểm vinid nhận ngay mã thẻ nạp tiền điện...,xXC3723,0
1,Buy 2 get 1,0,0,>voucher ưu đãi >khi mua 02 ly trà cỡ lớn (áp ...,MA_HIGHLANDS_01022024_SF4601,0
2,Free SKU,0,0,>voucher miễn phí combo cắt + hấp tóc trị giá ...,MA_YTOHAIRSALON_02062023_SF4718,0
3,Discount Fixed Amount (For Total Bill),300000,0,">voucher giảm 300,000đ cho hóa đơn 1,500,000đ ...",MAA_TORANO_24102023_SF4114,"1,500,000"
4,Discount Percentage (For SKU),0,5,>voucher ưu đãi giảm 5% cho đơn hàng bất kỳ kh...,MAA_VUANEM_01022024_SF4596,0
...,...,...,...,...,...,...
310,Free SKU,0,0,">tặng kem nền double wear 7ml trị giá 380,000đ...",MA_ESTEELAUDER_05032024_SF4664,0
311,Discount Percentage (For SKU),0,43,">combo giảm 43% chỉ còn 465,000đ (giá gốc 820,...",MA_BONGON555_29032023_SF4748,0
312,Cash Voucher,50000,0,công của >highlands coffee> là: không gian qu...,xXC8525,0
313,Discount Percentage (For SKU),0,30,>giảm 30% gói pinkcare - xét nghiệm tầm soát n...,MAA_GENSOLUTION_25122023_SF4416,0


In [124]:
pd.options.display.float_format = '{:,.2f}'.format

temp = df_merchant_sum_up.merge(df_scheme.loc[:,['VOUCHER_CODE','minimum_bill']], on = 'VOUCHER_CODE').groupby(['category','sub_category'],as_index = False).agg(
    avg_conditions = ('minimum_bill',lambda x: np.mean(x))
)

temp.loc[
    (temp['category'] == 'Ẩm thực') | (temp['sub_category'] == 'Viễn thông') ,
    :
        ]

,category,sub_category,avg_conditions
21,Thương mại điện tử,Viễn thông,"1,000.00"
26,Tiện ích - Nhà mạng,Viễn thông,"1,111.11"
31,Ẩm thực,Bệnh viện - Phòng khám,0.00
32,Ẩm thực,Cửa hàng kem/bánh/kẹo,0.00
33,Ẩm thực,Nhà hàng - Quán ăn,"102,894.74"
34,Ẩm thực,Đồ uống,"34,166.67"


In [114]:
df_scheme['minimum_bill'] = df_scheme['minimum_bill'].astype(int)

In [109]:
df_scheme['minimum_bill'] = df_scheme['minimum_bill'].apply(lambda x: 20000 if x == '20000 VND' else x)